Chaine de prétraitement des données

# Librairies

In [10]:
import pandas as pd
from PIL import Image
import numpy as np
import io
import os
from pathlib import Path
import random

import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split
from pyspark.sql import SparkSession

# Création du dossier de test1

On va sélectionner pour chaque catégorie d'image 3 images aléatoires et créer une arborescence.

Le dossier d'images utilisé pour créer Test1 sont celles de ce dossier : fruits-360_dataset\fruits-360\Test

## Paths

In [11]:
random.seed(42)

In [12]:
images_folder_filepath = "/home/maxime/projects/P9/data/Images/Test"
main_folder = Path(images_folder_filepath)

test1_folder_filepath = "/home/maxime/projects/P9/data/Images/Test/Test1"
test1_data_folder = Path(test1_folder_filepath)

IMAGE_EXTENSIONS = {".jpg", ".jpeg", ".png", ".gif", ".bmp"} # Définition des extensions d'aimages valides
NUM_IMAGES_TO_SELECT = 3
selected_image_paths = [] # La liste qui contiendra tous les chemins des images sélectionnées

## Sélection des images et de leurs classes

In [ ]:
if test1_data_folder.is_dir():
    print(f"Le dossier d'images Test1 existe déja")
else:
    print(f"Création du dossier d'images Test1")

    # On parcourt chaque élément (fichier ou dossier) dans le dossier principal
    for subfolder in main_folder.iterdir():
        
        # Vérifiez que l'élément est bien un dossier
        if subfolder.is_dir():
            print(f"--- Traitement du sous-dossier : {subfolder.name} ---")
            
            # Listez toutes les images valides dans ce sous-dossier
            images_in_subfolder = [
                item for item in subfolder.iterdir() 
                if item.is_file() and item.suffix.lower() in IMAGE_EXTENSIONS
            ]
            
            num_found = len(images_in_subfolder) #Permet de d'assurer qu'on a au moins 3 images à sélectionner dedans

            # S'il y a assez d'images, on en sélectionne 3 au hasard
            if num_found >= NUM_IMAGES_TO_SELECT:
                print(f"Trouvé {num_found} images. Sélection de {NUM_IMAGES_TO_SELECT} au hasard.")
                selected_samples = random.sample(images_in_subfolder, NUM_IMAGES_TO_SELECT)
            # Sinon, on sélectionne toutes les images disponibles et afficher un avertissement
            else:
                print(f"AVERTISSEMENT : Trouvé seulement {num_found} image(s). Sélection de toutes les images.")
                selected_samples = images_in_subfolder.copy()

            # Ajouter les chemins sélectionnés à notre liste finale
            # On utilise extend() pour ajouter tous les éléments de la liste 'selected_samples'
            selected_image_paths.extend(selected_samples)
            print("-" * (len(subfolder.name) + 30)) # Ligne de séparation pour la lisibilité

    # --- 3. Affichage des résultats ---

    print("\n--- Sélection des images pour le dossier Test1 terminée ---")
    print(f"Nombre total d'images sélectionnées : {len(selected_image_paths)}")

    # Affichons les 10 premiers chemins pour vérifier
    print("\nVoici les 10 premiers chemins sélectionnés :")
    for path in selected_image_paths[:10]:
        print(path)
    print(f"{len(selected_image_paths)} images ont été selectionnées")

    print("\nRécupération de la catégorie pour chaque image sélectionnée")

    loaded_images = []
    labels = []

    for path in selected_image_paths:
        try:
            # a) Extraire le nom du dossier parent comme classe (label)
            # Si le chemin est /.../chat/image01.jpg, path.parent.name sera "chat"
            label = path.parent.name
            
            # b) Charger l'image avec Pillow
            image = Image.open(path)
            
            # c) On va convertir l'image en mode 'RGB'
            # Cela garantit que toutes les images ont 3 canaux de couleur (Rouge, Vert, Bleu).
            # permet d'éviter les problèmes avec les images en noir et blanc (mode 'L') ou avec canal alpha (mode 'RGBA').
            image_rgb = image.convert('RGB')
            
            # d) Ajouter l'image chargée et son label aux listes
            loaded_images.append(image_rgb)
            labels.append(label)

        except Exception as e:
            # Gérer le cas où une image est corrompue ou ne peut être ouverte
            print(f"AVERTISSEMENT : Impossible de charger l'image {path}. Erreur : {e}")
            # On passe à l'image suivante sans planter le script
            continue

Création du dossier d'images Test1
--- Traitement du sous-dossier : Chestnut ---
Trouvé 153 images. Sélection de 3 au hasard.
--------------------------------------
--- Traitement du sous-dossier : Grape White ---
Trouvé 166 images. Sélection de 3 au hasard.
-----------------------------------------
--- Traitement du sous-dossier : Apple Red 2 ---
Trouvé 164 images. Sélection de 3 au hasard.
-----------------------------------------
--- Traitement du sous-dossier : Cucumber Ripe ---
Trouvé 130 images. Sélection de 3 au hasard.
-------------------------------------------
--- Traitement du sous-dossier : Nut Pecan ---
Trouvé 178 images. Sélection de 3 au hasard.
---------------------------------------
--- Traitement du sous-dossier : Dates ---
Trouvé 166 images. Sélection de 3 au hasard.
-----------------------------------
--- Traitement du sous-dossier : Tomato 1 ---
Trouvé 246 images. Sélection de 3 au hasard.
--------------------------------------
--- Traitement du sous-dossier : Kumq

## Dataframe contenant les paths et leurs classes

In [15]:
pd.set_option('max_colwidth', 400)

In [16]:
images_test1_df = pd.DataFrame({
    'image': loaded_images,
    'label': labels
})

images_test1_df

,image,label
0,<PIL.Image.Image image mode=RGB size=100x100 at 0x7F444A875010>,Chestnut
1,<PIL.Image.Image image mode=RGB size=100x100 at 0x7F444861E150>,Chestnut
2,<PIL.Image.Image image mode=RGB size=100x100 at 0x7F444861E420>,Chestnut
3,<PIL.Image.Image image mode=RGB size=100x100 at 0x7F444861FE90>,Grape White
4,<PIL.Image.Image image mode=RGB size=100x100 at 0x7F444861C110>,Grape White
...,...,...
388,<PIL.Image.Image image mode=RGB size=100x100 at 0x7F444858A6C0>,Apple Red 1
389,<PIL.Image.Image image mode=RGB size=100x100 at 0x7F444858A720>,Apple Red 1
390,<PIL.Image.Image image mode=RGB size=100x100 at 0x7F444858A780>,Pear Kaiser
391,<PIL.Image.Image image mode=RGB size=100x100 at 0x7F444858A7E0>,Pear Kaiser


# Création de la SparkSession

In [27]:
spark = (SparkSession
             .builder # .builder starts the construction process
             .appName('P9')
             .master('local') # .master specifies where the Spark application runs.
             .config("spark.sql.parquet.writeLegacyFormat", 'true')
             .getOrCreate() # If a SparkSession with the same configuration already exists, it returns that session. - If no session exists, it creates a new one based on the builder settings
)

25/07/02 05:43:31 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [30]:
spark

In [28]:
sc = spark.sparkContext

In [29]:
sc

<SparkContext master=local appName=P9>